# Building AI Agents with DSPy: Customer Service Agent

This notebook demonstrates how to build intelligent customer service agents using DSPy:
- Multi-step reasoning for customer queries
- Intent classification and response generation
- Tool integration for order management
- Memory and context management

We'll build a comprehensive customer service agent that can handle various customer scenarios.

## Setup and Imports

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
import json
from typing import List, Dict, Any, Optional
from datetime import datetime, timedelta
from utils import setup_default_lm, print_step, print_result, print_error
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../../.env')

## Configure Language Model

In [ ]:
print_step("Configuring Language Model", "Setting up DSPy with OpenAI")

try:
    lm = setup_default_lm(provider="openai", model="gpt-4o", max_tokens=1500)
    dspy.configure(lm=lm)
    print_result("Language model configured successfully!")
except Exception as e:
    print_error(f"Failed to configure language model: {e}")
    print("Make sure you have set your OPENAI_API_KEY in the .env file")

## Mock Backend Systems

Let's create mock systems for order management, customer data, and knowledge base.

In [ ]:
print_step("Creating Mock Backend Systems", "Setting up order management and customer data")

class MockOrderSystem:
    """Mock order management system."""
    
    def __init__(self):
        self.orders = {
            "ORD-12345": {
                "id": "ORD-12345",
                "customer_id": "CUST-001",
                "status": "shipped",
                "items": ["Laptop", "Mouse"],
                "total": 1299.99,
                "order_date": "2024-01-15",
                "tracking_number": "TRK-98765"
            },
            "ORD-67890": {
                "id": "ORD-67890",
                "customer_id": "CUST-002",
                "status": "processing",
                "items": ["Headphones"],
                "total": 199.99,
                "order_date": "2024-01-20",
                "tracking_number": None
            }
        }
    
    def get_order(self, order_id: str) -> Optional[Dict]:
        """Get order details by order ID."""
        return self.orders.get(order_id)
    
    def update_order_status(self, order_id: str, status: str) -> bool:
        """Update order status."""
        if order_id in self.orders:
            self.orders[order_id]["status"] = status
            return True
        return False
    
    def cancel_order(self, order_id: str) -> bool:
        """Cancel an order."""
        if order_id in self.orders and self.orders[order_id]["status"] in ["processing", "pending"]:
            self.orders[order_id]["status"] = "cancelled"
            return True
        return False

class MockCustomerDB:
    """Mock customer database."""
    
    def __init__(self):
        self.customers = {
            "john.doe@email.com": {
                "id": "CUST-001",
                "name": "John Doe",
                "email": "john.doe@email.com",
                "tier": "premium",
                "orders": ["ORD-12345"]
            },
            "jane.smith@email.com": {
                "id": "CUST-002",
                "name": "Jane Smith",
                "email": "jane.smith@email.com",
                "tier": "standard",
                "orders": ["ORD-67890"]
            }
        }
    
    def get_customer(self, email: str) -> Optional[Dict]:
        """Get customer details by email."""
        return self.customers.get(email)

class MockKnowledgeBase:
    """Mock knowledge base for FAQs and policies."""
    
    def __init__(self):
        self.articles = {
            "shipping_policy": "We offer free shipping on orders over $50. Standard shipping takes 3-5 business days.",
            "return_policy": "Items can be returned within 30 days of purchase for a full refund.",
            "warranty_info": "All electronics come with a 1-year manufacturer warranty.",
            "payment_methods": "We accept credit cards, PayPal, and bank transfers."
        }
    
    def search(self, query: str) -> str:
        """Search knowledge base for relevant information."""
        query_lower = query.lower()
        for key, content in self.articles.items():
            if any(word in query_lower for word in key.split('_')):
                return content
        return "I don't have specific information about that. Please contact our support team for more details."

# Initialize mock systems
order_system = MockOrderSystem()
customer_db = MockCustomerDB()
knowledge_base = MockKnowledgeBase()

print_result("Mock backend systems initialized!")

## Agent Signatures

Let's define the signatures our customer service agent will use.

In [ ]:
print_step("Defining Agent Signatures", "Creating input/output specifications")

class ClassifyIntent(dspy.Signature):
    """Classify the customer's intent from their message."""
    customer_message = dspy.InputField(desc="The customer's message or query")
    intent = dspy.OutputField(desc="Intent: order_inquiry, technical_support, billing_question, general_info, complaint, or other")
    confidence = dspy.OutputField(desc="Confidence level (high/medium/low)")

class ExtractOrderInfo(dspy.Signature):
    """Extract order-related information from customer message."""
    customer_message = dspy.InputField(desc="The customer's message")
    order_id = dspy.OutputField(desc="Order ID if mentioned, otherwise 'none'")
    email = dspy.OutputField(desc="Customer email if mentioned, otherwise 'none'")
    action_requested = dspy.OutputField(desc="What the customer wants to do with their order")

class GenerateResponse(dspy.Signature):
    """Generate a helpful customer service response."""
    customer_message = dspy.InputField(desc="The customer's original message")
    intent = dspy.InputField(desc="Classified intent")
    context_info = dspy.InputField(desc="Relevant information from systems (orders, customer data, etc.)")
    response = dspy.OutputField(desc="A helpful, professional, and empathetic response")
    suggested_actions = dspy.OutputField(desc="Suggested next steps or actions")

class EscalationCheck(dspy.Signature):
    """Determine if a customer inquiry should be escalated to human support."""
    customer_message = dspy.InputField(desc="The customer's message")
    intent = dspy.InputField(desc="Classified intent")
    previous_interactions = dspy.InputField(desc="Summary of previous interactions")
    should_escalate = dspy.OutputField(desc="true if should escalate to human, false otherwise")
    escalation_reason = dspy.OutputField(desc="Reason for escalation if applicable")

print_result("Agent signatures defined successfully!")

## Customer Service Agent

Now let's build the main customer service agent that orchestrates all components.

In [ ]:
print_step("Creating Customer Service Agent", "Building the intelligent agent")

class CustomerServiceAgent(dspy.Module):
    """Intelligent customer service agent using DSPy."""
    
    def __init__(self, order_system, customer_db, knowledge_base):
        super().__init__()
        
        # Backend systems
        self.order_system = order_system
        self.customer_db = customer_db
        self.knowledge_base = knowledge_base
        
        # DSPy modules
        self.classify_intent = dspy.Predict(ClassifyIntent)
        self.extract_order_info = dspy.Predict(ExtractOrderInfo)
        self.generate_response = dspy.ChainOfThought(GenerateResponse)
        self.check_escalation = dspy.Predict(EscalationCheck)
        
        # Conversation memory
        self.conversation_history = []
    
    def get_order_context(self, order_id: str, email: str) -> str:
        """Get order and customer context information."""
        context_parts = []
        
        # Get order info
        if order_id != 'none':
            order = self.order_system.get_order(order_id)
            if order:
                context_parts.append(f"Order {order_id}: Status={order['status']}, Items={order['items']}, Total=${order['total']}, Date={order['order_date']}")
            else:
                context_parts.append(f"Order {order_id} not found in system.")
        
        # Get customer info
        if email != 'none':
            customer = self.customer_db.get_customer(email)
            if customer:
                context_parts.append(f"Customer: {customer['name']} ({customer['tier']} tier), Orders: {customer['orders']}")
            else:
                context_parts.append(f"Customer with email {email} not found.")
        
        return " | ".join(context_parts) if context_parts else "No specific order or customer context available."
    
    def handle_order_action(self, order_id: str, action: str) -> str:
        """Handle order-related actions."""
        if "cancel" in action.lower():
            if self.order_system.cancel_order(order_id):
                return f"Order {order_id} has been successfully cancelled."
            else:
                return f"Unable to cancel order {order_id}. It may already be shipped or completed."
        
        return "Action processed."
    
    def forward(self, customer_message: str, customer_email: str = None):
        """Process customer message and generate response."""
        
        # Step 1: Classify intent
        intent_result = self.classify_intent(customer_message=customer_message)
        
        # Step 2: Extract order information if relevant
        order_info = self.extract_order_info(customer_message=customer_message)
        
        # Step 3: Gather context information
        context_info = self.get_order_context(order_info.order_id, order_info.email or customer_email or 'none')
        
        # Step 4: Add knowledge base information for general questions
        if intent_result.intent in ['general_info', 'technical_support']:
            kb_info = self.knowledge_base.search(customer_message)
            context_info += f" | Knowledge Base: {kb_info}"
        
        # Step 5: Handle order actions if needed
        action_result = ""
        if intent_result.intent == 'order_inquiry' and order_info.order_id != 'none':
            action_result = self.handle_order_action(order_info.order_id, order_info.action_requested)
            context_info += f" | Action Result: {action_result}"
        
        # Step 6: Generate response
        response_result = self.generate_response(
            customer_message=customer_message,
            intent=intent_result.intent,
            context_info=context_info
        )
        
        # Step 7: Check for escalation
        escalation_result = self.check_escalation(
            customer_message=customer_message,
            intent=intent_result.intent,
            previous_interactions=str(self.conversation_history[-3:])  # Last 3 interactions
        )
        
        # Step 8: Update conversation history
        interaction = {
            "timestamp": datetime.now().isoformat(),
            "customer_message": customer_message,
            "intent": intent_result.intent,
            "response": response_result.response
        }
        self.conversation_history.append(interaction)
        
        return dspy.Prediction(
            intent=intent_result.intent,
            confidence=intent_result.confidence,
            order_id=order_info.order_id,
            extracted_email=order_info.email,
            action_requested=order_info.action_requested,
            context_info=context_info,
            reasoning=response_result.reasoning,
            response=response_result.response,
            suggested_actions=response_result.suggested_actions,
            should_escalate=escalation_result.should_escalate,
            escalation_reason=escalation_result.escalation_reason,
            action_result=action_result
        )

# Create the customer service agent
agent = CustomerServiceAgent(order_system, customer_db, knowledge_base)

print_result("Customer service agent created successfully!")

## Test Scenarios

Let's test our customer service agent with various scenarios.

In [ ]:
print_step("Testing Customer Service Agent", "Running various customer scenarios")

def test_agent_scenario(message: str, email: str = None, scenario_name: str = ""):
    """Test the agent with a specific scenario."""
    print(f"\n🎯 Scenario: {scenario_name}")
    print(f"Customer Message: {message}")
    if email:
        print(f"Customer Email: {email}")
    print("-" * 60)
    
    result = agent(customer_message=message, customer_email=email)
    
    print(f"Intent: {result.intent} (Confidence: {result.confidence})")
    print(f"Response: {result.response}")
    print(f"Suggested Actions: {result.suggested_actions}")
    
    if result.should_escalate == "true":
        print(f"🚨 ESCALATION NEEDED: {result.escalation_reason}")
    
    if result.action_result:
        print(f"Action Performed: {result.action_result}")
    
    print("=" * 60)

# Test scenarios
test_scenarios = [
    {
        "message": "Hi, I want to check the status of my order ORD-12345",
        "email": "john.doe@email.com",
        "scenario": "Order Status Inquiry"
    },
    {
        "message": "I need to cancel my order ORD-67890 immediately!",
        "email": "jane.smith@email.com",
        "scenario": "Order Cancellation Request"
    },
    {
        "message": "What's your return policy? I'm not happy with my purchase.",
        "scenario": "Policy Question"
    },
    {
        "message": "This is terrible! I've been waiting 3 weeks and still no product! I want my money back NOW!",
        "scenario": "Angry Customer Complaint"
    },
    {
        "message": "Do you offer free shipping? I'm thinking of placing a large order.",
        "scenario": "Shipping Inquiry"
    }
]

for scenario in test_scenarios:
    test_agent_scenario(
        message=scenario["message"],
        email=scenario.get("email"),
        scenario_name=scenario["scenario"]
    )

## Conversation Flow

Let's test a multi-turn conversation to see how the agent handles context.

In [ ]:
print_step("Testing Conversation Flow", "Multi-turn conversation with context")

def simulate_conversation(messages: List[str], customer_email: str = None):
    """Simulate a multi-turn conversation."""
    print(f"\n💬 Starting conversation with {customer_email or 'anonymous customer'}")
    print("=" * 70)
    
    for i, message in enumerate(messages, 1):
        print(f"\nTurn {i}:")
        print(f"Customer: {message}")
        
        result = agent(customer_message=message, customer_email=customer_email)
        
        print(f"Agent ({result.intent}): {result.response}")
        
        if result.should_escalate == "true":
            print(f"🚨 [ESCALATION TRIGGERED: {result.escalation_reason}]")
        
        print("-" * 50)

# Simulate a conversation
conversation_messages = [
    "Hi, I placed an order last week but haven't received any updates",
    "My order number is ORD-12345",
    "Great! When will it arrive?",
    "Actually, I need to change my delivery address. Is that possible?"
]

simulate_conversation(conversation_messages, "john.doe@email.com")

## Agent Analytics

Let's analyze the agent's performance and conversation history.

In [ ]:
print_step("Agent Analytics", "Analyzing conversation history and performance")

def analyze_agent_performance():
    """Analyze the agent's conversation history."""
    history = agent.conversation_history
    
    if not history:
        print("No conversation history available.")
        return
    
    # Intent distribution
    intents = [interaction['intent'] for interaction in history]
    intent_counts = {}
    for intent in intents:
        intent_counts[intent] = intent_counts.get(intent, 0) + 1
    
    print(f"Total Interactions: {len(history)}")
    print(f"Intent Distribution: {intent_counts}")
    
    # Recent interactions
    print("\nRecent Interactions:")
    for interaction in history[-3:]:
        print(f"- {interaction['timestamp']}: {interaction['intent']} - {interaction['customer_message'][:50]}...")
    
    # Time analysis
    if len(history) > 1:
        first_time = datetime.fromisoformat(history[0]['timestamp'])
        last_time = datetime.fromisoformat(history[-1]['timestamp'])
        duration = last_time - first_time
        print(f"\nConversation Duration: {duration.total_seconds():.1f} seconds")

analyze_agent_performance()

# Show full conversation history
print("\n📋 Full Conversation History:")
for i, interaction in enumerate(agent.conversation_history, 1):
    print(f"{i}. [{interaction['intent']}] {interaction['customer_message']}")
    print(f"   Response: {interaction['response'][:100]}...")
    print()

## Advanced Features Demo

Let's demonstrate some advanced features like sentiment analysis and proactive suggestions.

In [ ]:
print_step("Advanced Features", "Sentiment analysis and proactive suggestions")

class SentimentAnalysis(dspy.Signature):
    """Analyze the sentiment and emotional tone of customer message."""
    customer_message = dspy.InputField(desc="The customer's message")
    sentiment = dspy.OutputField(desc="Sentiment: positive, neutral, negative, or frustrated")
    emotion_level = dspy.OutputField(desc="Emotion intensity: low, medium, or high")
    key_concerns = dspy.OutputField(desc="Main concerns or issues mentioned")

class ProactiveSuggestion(dspy.Signature):
    """Generate proactive suggestions for customer based on their history and current context."""
    customer_context = dspy.InputField(desc="Customer information and order history")
    current_interaction = dspy.InputField(desc="Current customer message and intent")
    suggestions = dspy.OutputField(desc="Proactive suggestions or offers for the customer")
    reasoning = dspy.OutputField(desc="Why these suggestions are relevant")

# Create enhanced agent modules
sentiment_analyzer = dspy.Predict(SentimentAnalysis)
suggestion_generator = dspy.ChainOfThought(ProactiveSuggestion)

def enhanced_agent_analysis(message: str, customer_email: str = None):
    """Enhanced agent with sentiment analysis and proactive suggestions."""
    print(f"\n🔍 Enhanced Analysis for: {message}")
    print("-" * 60)
    
    # Sentiment analysis
    sentiment_result = sentiment_analyzer(customer_message=message)
    print(f"Sentiment: {sentiment_result.sentiment} (Intensity: {sentiment_result.emotion_level})")
    print(f"Key Concerns: {sentiment_result.key_concerns}")
    
    # Get customer context
    customer_context = "No customer context available"
    if customer_email:
        customer = customer_db.get_customer(customer_email)
        if customer:
            customer_context = f"Customer: {customer['name']} ({customer['tier']} tier), Previous orders: {customer['orders']}"
    
    # Generate proactive suggestions
    suggestion_result = suggestion_generator(
        customer_context=customer_context,
        current_interaction=f"Message: {message}, Sentiment: {sentiment_result.sentiment}"
    )
    
    print(f"\n💡 Proactive Suggestions:")
    print(f"Suggestions: {suggestion_result.suggestions}")
    print(f"Reasoning: {suggestion_result.reasoning}")
    
    print("=" * 60)

# Test enhanced features
enhanced_test_messages = [
    ("I'm really disappointed with the quality of my recent purchase", "john.doe@email.com"),
    ("Thank you so much for the excellent service! Very happy with my order", "jane.smith@email.com"),
    ("I've been a loyal customer for years but this experience has been terrible", "john.doe@email.com")
]

for message, email in enhanced_test_messages:
    enhanced_agent_analysis(message, email)

## Summary

In this notebook, we built a comprehensive customer service agent using DSPy that includes:

### Core Components:
1. **Intent Classification** - Automatically categorizes customer inquiries
2. **Information Extraction** - Pulls relevant details like order IDs and emails
3. **Context Integration** - Connects with backend systems (orders, customers, knowledge base)
4. **Response Generation** - Creates helpful, contextual responses
5. **Escalation Logic** - Determines when human intervention is needed

### Advanced Features:
- **Conversation Memory** - Tracks interaction history for context
- **Multi-turn Conversations** - Handles complex dialogue flows
- **Sentiment Analysis** - Understands customer emotional state
- **Proactive Suggestions** - Offers relevant help based on context
- **Analytics Dashboard** - Monitors agent performance

### Key Benefits:
- **Scalable**: Can handle multiple customer inquiries simultaneously
- **Consistent**: Provides uniform service quality
- **Intelligent**: Makes informed decisions based on context
- **Trainable**: Can be optimized with DSPy optimizers
- **Integrated**: Works with existing business systems

This agent demonstrates how DSPy can be used to build sophisticated AI applications that combine multiple reasoning steps, external system integration, and adaptive behavior based on context and history.

Next steps could include:
- Training with real customer service data
- Implementing more sophisticated retrieval systems
- Adding voice/chat interface capabilities
- Integrating with actual CRM and order management systems
- Optimizing with DSPy's optimization algorithms